# IO Latency

In [1]:
--	How to examine IO subsystem latencies from within SQL Server (Disk Latency)
	--	https://www.sqlskills.com/blogs/paul/how-to-examine-io-subsystem-latencies-from-within-sql-server/
	--	https://sqlperformance.com/2015/03/io-subsystem/monitoring-read-write-latency
	--	https://www.brentozar.com/blitz/slow-storage-reads-writes/

select	[Startup Time] = cast(create_date as smalldatetime), 
		[Uptime- Days HH:mm:ss)] = convert(varchar,datediff(day,'1900-01-01',(GETDATE()-create_date)))+' Days '+convert(varchar,(GETDATE()-create_date),108)
from sys.databases d where d.name = 'tempdb';

IF OBJECT_ID('tempdb..#VirtualFileStats') IS NOT NULL
	DROP TABLE #VirtualFileStats;
SELECT  
    [ReadLatency] =
        CASE WHEN [num_of_reads] = 0
            THEN 0 ELSE ([io_stall_read_ms] / [num_of_reads]) END,
    [WriteLatency] =
        CASE WHEN [num_of_writes] = 0
            THEN 0 ELSE ([io_stall_write_ms] / [num_of_writes]) END,
    [Latency] =
        CASE WHEN ([num_of_reads] = 0 AND [num_of_writes] = 0)
            THEN 0 ELSE ([io_stall] / ([num_of_reads] + [num_of_writes])) END,
    [AvgBPerRead] =
        CASE WHEN [num_of_reads] = 0
            THEN 0 ELSE ([num_of_bytes_read] / [num_of_reads]) END,
    [AvgBPerWrite] =
        CASE WHEN [num_of_writes] = 0
            THEN 0 ELSE ([num_of_bytes_written] / [num_of_writes]) END,
    [AvgBPerTransfer] =
        CASE WHEN ([num_of_reads] = 0 AND [num_of_writes] = 0)
            THEN 0 ELSE
                (([num_of_bytes_read] + [num_of_bytes_written]) /
                ([num_of_reads] + [num_of_writes])) END,
    LEFT ([mf].[physical_name], 2) AS [Drive],
    DB_NAME ([vfs].[database_id]) AS [DB],
    [FileLocation] = [mf].[physical_name],
	[DataReadGB] = convert(numeric(20,2),[num_of_bytes_read]*1.0/1024/1024/1024), 
	[DataWrittenGB] = convert(numeric(20,2),[num_of_bytes_written]*1.0/1024/1024/1024), 
	[NoOfReads] = [num_of_reads], [NoOfWrites] = [num_of_writes]
INTO #VirtualFileStats
FROM
    sys.dm_io_virtual_file_stats (NULL,NULL) AS [vfs]
JOIN sys.master_files AS [mf]
    ON [vfs].[database_id] = [mf].[database_id]
    AND [vfs].[file_id] = [mf].[file_id];

SELECT	top 20
		RunningQuery = 'TempDb-Latency',
		ReadLatency, WriteLatency, Latency, 
		Drive, DB, [FileLocation], DataReadGB, DataWrittenGB, [NoOfReads], [NoOfWrites],
		[Avg-KB-Per-Read] = convert(numeric(20,2),AvgBPerRead*1.0/1024),
		[Avg-KB-Per-Write] = convert(numeric(20,2),AvgBPerWrite*1.0/1024)
FROM #VirtualFileStats
WHERE [DB] = 'tempdb'
ORDER BY ([ReadLatency]+[WriteLatency]) DESC;

SELECT	top 20
		RunningQuery = 'Top-20-UserDb-Latency',
		ReadLatency, WriteLatency, Latency, 
		Drive, DB, [FileLocation], DataReadGB, DataWrittenGB, [NoOfReads], [NoOfWrites],
		[Avg-KB-Per-Read] = convert(numeric(20,2),AvgBPerRead*1.0/1024),
		[Avg-KB-Per-Write] = convert(numeric(20,2),AvgBPerWrite*1.0/1024)
FROM #VirtualFileStats
WHERE [DB] <> 'tempdb'
ORDER BY ([ReadLatency]+[WriteLatency]) DESC;

SELECT	top 20
		RunningQuery = 'top-20-DbFiles-By-IO',		
		Drive, DB, [FileLocation], DataReadGB, DataWrittenGB, 
		ReadLatency, WriteLatency, Latency, [NoOfReads], [NoOfWrites],
		[Avg-KB-Per-Read] = convert(numeric(20,2),AvgBPerRead*1.0/1024),
		[Avg-KB-Per-Write] = convert(numeric(20,2),AvgBPerWrite*1.0/1024)
FROM #VirtualFileStats
ORDER BY (DataReadGB+DataWrittenGB) DESC;
GO

Error: None

Total execution time: 00:00:14.7214102